<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/263_MissionOrchestratorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading utilities for Mission Orchestrator Agent

In [ ]:
"""Data loading utilities for Mission Orchestrator Agent"""

import json
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from config import MissionOrchestratorConfig


def _get_data_path(filename: str, config: Optional[MissionOrchestratorConfig] = None) -> Path:
    """Get full path to data file"""
    if config is None:
        config = MissionOrchestratorConfig()
    data_dir = Path(__file__).parent.parent.parent.parent / config.data_dir
    return data_dir / filename


def load_business_missions(config: Optional[MissionOrchestratorConfig] = None) -> List[Dict[str, Any]]:
    """Load all business missions from JSON file"""
    file_path = _get_data_path("business_missions.json", config)
    with open(file_path, 'r') as f:
        return json.load(f)


def load_mission_tasks(mission_id: str, config: Optional[MissionOrchestratorConfig] = None) -> List[Dict[str, Any]]:
    """Load tasks for a specific mission"""
    file_path = _get_data_path("decomposed_mission_tasks.json", config)
    with open(file_path, 'r') as f:
        all_mission_tasks = json.load(f)

    # Find tasks for the specific mission
    for mission_data in all_mission_tasks:
        if mission_data.get("mission_id") == mission_id:
            return mission_data.get("tasks", [])

    return []


def load_specialized_agents(config: Optional[MissionOrchestratorConfig] = None) -> List[Dict[str, Any]]:
    """Load all specialized agents from JSON file"""
    file_path = _get_data_path("specialized_agents.json", config)
    with open(file_path, 'r') as f:
        return json.load(f)


def load_agent_capabilities_matrix(config: Optional[MissionOrchestratorConfig] = None) -> List[Dict[str, Any]]:
    """Load agent capabilities matrix from JSON file"""
    file_path = _get_data_path("agent_capabilities_matrix.json", config)
    with open(file_path, 'r') as f:
        return json.load(f)


def load_mission_kpis(mission_id: str, config: Optional[MissionOrchestratorConfig] = None) -> Dict[str, Any]:
    """Load KPIs for a specific mission"""
    file_path = _get_data_path("mission_kpis.json", config)
    with open(file_path, 'r') as f:
        all_mission_kpis = json.load(f)

    # Find KPIs for the specific mission
    for mission_kpi_data in all_mission_kpis:
        if mission_kpi_data.get("mission_id") == mission_id:
            return mission_kpi_data.get("kpis", {})

    return {}


def get_mission_by_id(mission_id: str, config: Optional[MissionOrchestratorConfig] = None) -> Optional[Dict[str, Any]]:
    """Get a specific mission by ID"""
    missions = load_business_missions(config)
    for mission in missions:
        if mission.get("mission_id") == mission_id:
            return mission
    return None


def get_task_by_id(task_id: str, config: Optional[MissionOrchestratorConfig] = None) -> Optional[Dict[str, Any]]:
    """Get a specific task by ID across all missions"""
    file_path = _get_data_path("decomposed_mission_tasks.json", config)
    with open(file_path, 'r') as f:
        all_mission_tasks = json.load(f)

    # Search across all missions
    for mission_data in all_mission_tasks:
        tasks = mission_data.get("tasks", [])
        for task in tasks:
            if task.get("task_id") == task_id:
                return task
    return None


def get_agent_by_id(agent_id: str, config: Optional[MissionOrchestratorConfig] = None) -> Optional[Dict[str, Any]]:
    """Get a specific agent by ID"""
    agents = load_specialized_agents(config)
    for agent in agents:
        if agent.get("agent_id") == agent_id:
            return agent
    return None


def get_capable_agents_for_task(task_id: str, config: Optional[MissionOrchestratorConfig] = None) -> List[str]:
    """Get list of agent IDs capable of performing a task"""
    matrix = load_agent_capabilities_matrix(config)
    for mapping in matrix:
        if mapping.get("task_id") == task_id:
            return mapping.get("capable_agents", [])
    return []


def validate_data_consistency(config: Optional[MissionOrchestratorConfig] = None) -> Tuple[bool, List[str]]:
    """
    Validate data consistency across all files.

    Returns:
        (is_valid, errors): Tuple of validation status and list of error messages
    """
    errors = []

    # Load all data
    missions = load_business_missions(config)
    agents = load_specialized_agents(config)
    matrix = load_agent_capabilities_matrix(config)

    # Get all mission IDs
    mission_ids = {m.get("mission_id") for m in missions if m.get("mission_id")}

    # Get all agent IDs
    agent_ids = {a.get("agent_id") for a in agents if a.get("agent_id")}

    # Get all task IDs
    file_path = _get_data_path("decomposed_mission_tasks.json", config)
    with open(file_path, 'r') as f:
        all_mission_tasks = json.load(f)

    task_ids = set()
    for mission_data in all_mission_tasks:
        tasks = mission_data.get("tasks", [])
        for task in tasks:
            task_id = task.get("task_id")
            if task_id:
                task_ids.add(task_id)

    # Validate: Each mission has KPIs
    file_path = _get_data_path("mission_kpis.json", config)
    with open(file_path, 'r') as f:
        all_mission_kpis = json.load(f)

    kpi_mission_ids = {kpi.get("mission_id") for kpi in all_mission_kpis if kpi.get("mission_id")}
    missing_kpis = mission_ids - kpi_mission_ids
    if missing_kpis:
        errors.append(f"Missions without KPIs: {missing_kpis}")

    # Validate: Each task in matrix has a corresponding task definition
    matrix_task_ids = {m.get("task_id") for m in matrix if m.get("task_id")}
    missing_tasks = matrix_task_ids - task_ids
    if missing_tasks:
        errors.append(f"Tasks in matrix without definition: {missing_tasks}")

    # Validate: Each task has at least one capable agent
    for mapping in matrix:
        task_id = mapping.get("task_id")
        capable_agents = mapping.get("capable_agents", [])
        if not capable_agents:
            errors.append(f"Task {task_id} has no capable agents")

    # Validate: All agents in matrix exist
    matrix_agent_ids = set()
    for mapping in matrix:
        matrix_agent_ids.update(mapping.get("capable_agents", []))

    missing_agents = matrix_agent_ids - agent_ids
    if missing_agents:
        errors.append(f"Agents in matrix that don't exist: {missing_agents}")

    is_valid = len(errors) == 0
    return (is_valid, errors)



# Tests for data loading utilities

In [ ]:
"""Tests for data loading utilities"""

import pytest
from agents.mission_orchestrator.utilities.data_loading import (
    load_business_missions,
    load_mission_tasks,
    load_specialized_agents,
    load_agent_capabilities_matrix,
    load_mission_kpis,
    get_mission_by_id,
    get_task_by_id,
    get_agent_by_id,
    get_capable_agents_for_task,
    validate_data_consistency
)


def test_load_business_missions():
    """Test loading all business missions"""
    missions = load_business_missions()
    assert len(missions) > 0
    assert "mission_id" in missions[0]
    assert "mission_name" in missions[0]
    assert "description" in missions[0]


def test_get_mission_by_id():
    """Test getting a specific mission by ID"""
    mission = get_mission_by_id("M001")
    assert mission is not None
    assert mission["mission_id"] == "M001"
    assert "mission_name" in mission


def test_get_mission_by_id_not_found():
    """Test getting a non-existent mission"""
    mission = get_mission_by_id("M999")
    assert mission is None


def test_load_mission_tasks():
    """Test loading tasks for a specific mission"""
    tasks = load_mission_tasks("M001")
    assert len(tasks) > 0
    assert "task_id" in tasks[0]
    assert "task" in tasks[0]
    assert "order" in tasks[0]
    assert "depends_on" in tasks[0]
    assert "estimated_duration_minutes" in tasks[0]
    assert "requires_human_approval" in tasks[0]


def test_load_mission_tasks_not_found():
    """Test loading tasks for non-existent mission"""
    tasks = load_mission_tasks("M999")
    assert tasks == []


def test_load_specialized_agents():
    """Test loading all specialized agents"""
    agents = load_specialized_agents()
    assert len(agents) > 0
    assert "agent_id" in agents[0]
    assert "name" in agents[0]
    assert "description" in agents[0]


def test_get_agent_by_id():
    """Test getting a specific agent by ID"""
    agent = get_agent_by_id("A1")
    assert agent is not None
    assert agent["agent_id"] == "A1"
    assert "name" in agent


def test_get_agent_by_id_not_found():
    """Test getting a non-existent agent"""
    agent = get_agent_by_id("A99")
    assert agent is None


def test_load_agent_capabilities_matrix():
    """Test loading capabilities matrix"""
    matrix = load_agent_capabilities_matrix()
    assert len(matrix) > 0
    assert "task_id" in matrix[0]
    assert "capable_agents" in matrix[0]
    assert isinstance(matrix[0]["capable_agents"], list)


def test_get_capable_agents_for_task():
    """Test getting capable agents for a task"""
    agent_ids = get_capable_agents_for_task("T1")
    assert len(agent_ids) > 0
    assert isinstance(agent_ids, list)
    assert "A1" in agent_ids  # Based on our data, T1 should map to A1


def test_get_capable_agents_for_task_not_found():
    """Test getting agents for non-existent task"""
    agent_ids = get_capable_agents_for_task("T99")
    assert agent_ids == []


def test_load_mission_kpis():
    """Test loading KPIs for a specific mission"""
    kpis = load_mission_kpis("M001")
    assert len(kpis) > 0
    # M001 should have onboarding time KPIs
    assert "target_onboarding_time_days" in kpis or "baseline_onboarding_time_days" in kpis


def test_load_mission_kpis_not_found():
    """Test loading KPIs for non-existent mission"""
    kpis = load_mission_kpis("M999")
    assert kpis == {}


def test_get_task_by_id():
    """Test getting a specific task by ID"""
    task = get_task_by_id("T1")
    assert task is not None
    assert task["task_id"] == "T1"
    assert "task" in task
    assert "order" in task


def test_get_task_by_id_not_found():
    """Test getting a non-existent task"""
    task = get_task_by_id("T99")
    assert task is None


def test_validate_data_consistency():
    """Test data consistency validation"""
    is_valid, errors = validate_data_consistency()
    # Our data should be consistent
    assert isinstance(is_valid, bool)
    assert isinstance(errors, list)
    # If there are errors, they should be informative
    if not is_valid:
        assert len(errors) > 0
        for error in errors:
            assert isinstance(error, str)
            assert len(error) > 0



# Test Data Loading

In [ ]:
def test_data_loading_node_success():
    """Test data loading node with valid mission_id"""
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    result = data_loading_node(state)

    assert "mission" in result
    assert "mission_tasks" in result
    assert "mission_kpis" in result
    assert "specialized_agents" in result
    assert "capabilities_matrix" in result

    assert result["mission"]["mission_id"] == "M001"
    assert len(result["mission_tasks"]) > 0
    assert len(result["specialized_agents"]) > 0
    assert len(result["capabilities_matrix"]) > 0


def test_data_loading_node_missing_mission_id():
    """Test data loading node with missing mission_id"""
    state: MissionOrchestratorState = {
        "errors": []
    }

    result = data_loading_node(state)

    assert "mission" not in result
    assert len(result.get("errors", [])) > 0
    assert "mission_id is required" in result["errors"][0]


def test_data_loading_node_invalid_mission_id():
    """Test data loading node with non-existent mission_id"""
    state: MissionOrchestratorState = {
        "mission_id": "M999",
        "errors": []
    }

    result = data_loading_node(state)

    assert "mission" not in result
    assert len(result.get("errors", [])) > 0
    assert "Mission M999 not found" in result["errors"][0]


def test_goal_planning_data_loading_flow():
    """Test complete flow: goal → planning → data loading"""
    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Execute goal node
    goal_result = goal_node(state)
    state = {**state, **goal_result}
    assert "goal" in state

    # Execute planning node
    planning_result = planning_node(state)
    state = {**state, **planning_result}
    assert "plan" in state

    # Execute data loading node
    data_result = data_loading_node(state)
    state = {**state, **data_result}

    assert "mission" in state
    assert "mission_tasks" in state
    assert "mission_kpis" in state
    assert "specialized_agents" in state
    assert "capabilities_matrix" in state
    assert state["mission"]["mission_id"] == "M001"
    assert len(state["mission_tasks"]) == 3  # M001 has 3 tasks

# Standalone test script for data loading utilities and node

In [ ]:
"""Standalone test script for data loading utilities and node"""

from agents.mission_orchestrator.utilities.data_loading import (
    load_business_missions,
    load_mission_tasks,
    load_specialized_agents,
    load_agent_capabilities_matrix,
    load_mission_kpis,
    get_mission_by_id,
    get_task_by_id,
    get_agent_by_id,
    get_capable_agents_for_task,
    validate_data_consistency
)
from agents.mission_orchestrator.nodes import data_loading_node
from config import MissionOrchestratorState


def test_utilities():
    """Test all data loading utilities"""
    print("=" * 60)
    print("Testing Data Loading Utilities")
    print("=" * 60)

    # Test 1: Load business missions
    print("\n1. Loading business missions...")
    missions = load_business_missions()
    print(f"   ✓ Loaded {len(missions)} missions")
    for mission in missions:
        print(f"   - {mission['mission_id']}: {mission['mission_name']}")

    # Test 2: Get specific mission
    print("\n2. Getting mission M001...")
    mission = get_mission_by_id("M001")
    if mission:
        print(f"   ✓ Found: {mission['mission_name']}")
        print(f"   Description: {mission['description']}")
    else:
        print("   ✗ Mission not found")

    # Test 3: Load mission tasks
    print("\n3. Loading tasks for mission M001...")
    tasks = load_mission_tasks("M001")
    print(f"   ✓ Loaded {len(tasks)} tasks")
    for task in tasks:
        print(f"   - {task['task_id']}: {task['task']} (order: {task['order']}, depends_on: {task['depends_on']})")

    # Test 4: Get specific task
    print("\n4. Getting task T1...")
    task = get_task_by_id("T1")
    if task:
        print(f"   ✓ Found: {task['task']}")
        print(f"   Duration: {task['estimated_duration_minutes']} minutes")
        print(f"   Requires approval: {task['requires_human_approval']}")
    else:
        print("   ✗ Task not found")

    # Test 5: Load specialized agents
    print("\n5. Loading specialized agents...")
    agents = load_specialized_agents()
    print(f"   ✓ Loaded {len(agents)} agents")
    for agent in agents:
        print(f"   - {agent['agent_id']}: {agent['name']}")

    # Test 6: Get specific agent
    print("\n6. Getting agent A1...")
    agent = get_agent_by_id("A1")
    if agent:
        print(f"   ✓ Found: {agent['name']}")
        print(f"   Description: {agent['description']}")
    else:
        print("   ✗ Agent not found")

    # Test 7: Load capabilities matrix
    print("\n7. Loading capabilities matrix...")
    matrix = load_agent_capabilities_matrix()
    print(f"   ✓ Loaded {len(matrix)} task-to-agent mappings")
    for mapping in matrix[:3]:  # Show first 3
        print(f"   - Task {mapping['task_id']} → Agents: {mapping['capable_agents']}")

    # Test 8: Get capable agents for task
    print("\n8. Getting capable agents for task T1...")
    agent_ids = get_capable_agents_for_task("T1")
    print(f"   ✓ Task T1 can be performed by: {agent_ids}")

    # Test 9: Load mission KPIs
    print("\n9. Loading KPIs for mission M001...")
    kpis = load_mission_kpis("M001")
    print(f"   ✓ Loaded KPIs:")
    for key, value in kpis.items():
        print(f"   - {key}: {value}")

    # Test 10: Validate data consistency
    print("\n10. Validating data consistency...")
    is_valid, errors = validate_data_consistency()
    if is_valid:
        print("   ✓ All data is consistent!")
    else:
        print(f"   ✗ Found {len(errors)} issues:")
        for error in errors:
            print(f"     - {error}")

    print("\n" + "=" * 60)
    print("Utility Tests Complete!")
    print("=" * 60)


def test_data_loading_node():
    """Test the data loading node"""
    print("\n" + "=" * 60)
    print("Testing Data Loading Node")
    print("=" * 60)

    # Test with valid mission_id
    print("\n1. Testing with mission_id M001...")
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    result = data_loading_node(state)

    if "mission" in result:
        print("   ✓ Node executed successfully")
        print(f"   - Mission: {result['mission']['mission_name']}")
        print(f"   - Tasks loaded: {len(result['mission_tasks'])}")
        print(f"   - Agents loaded: {len(result['specialized_agents'])}")
        print(f"   - Capabilities mappings: {len(result['capabilities_matrix'])}")
        print(f"   - KPIs loaded: {len(result['mission_kpis'])} keys")

        if result.get("errors"):
            print(f"   ⚠ Warnings: {len(result['errors'])}")
            for error in result["errors"]:
                print(f"     - {error}")
    else:
        print("   ✗ Node failed")
        if result.get("errors"):
            for error in result["errors"]:
                print(f"     - {error}")

    # Test with invalid mission_id
    print("\n2. Testing with invalid mission_id M999...")
    state = {
        "mission_id": "M999",
        "errors": []
    }

    result = data_loading_node(state)

    if "mission" not in result:
        print("   ✓ Correctly handled invalid mission_id")
        if result.get("errors"):
            print(f"   Error: {result['errors'][0]}")
    else:
        print("   ✗ Should have failed for invalid mission_id")

    # Test with missing mission_id
    print("\n3. Testing with missing mission_id...")
    state = {
        "errors": []
    }

    result = data_loading_node(state)

    if "mission" not in result:
        print("   ✓ Correctly handled missing mission_id")
        if result.get("errors"):
            print(f"   Error: {result['errors'][0]}")
    else:
        print("   ✗ Should have failed for missing mission_id")

    print("\n" + "=" * 60)
    print("Node Tests Complete!")
    print("=" * 60)


def test_full_flow():
    """Test the full flow: goal → planning → data loading"""
    print("\n" + "=" * 60)
    print("Testing Full Flow: Goal → Planning → Data Loading")
    print("=" * 60)

    from agents.mission_orchestrator.nodes import goal_node, planning_node

    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Step 1: Goal node
    print("\n1. Executing goal node...")
    goal_result = goal_node(state)
    state = {**state, **goal_result}
    if "goal" in state:
        print(f"   ✓ Goal defined: {state['goal']['objective']}")
    else:
        print("   ✗ Goal node failed")
        return

    # Step 2: Planning node
    print("\n2. Executing planning node...")
    planning_result = planning_node(state)
    state = {**state, **planning_result}
    if "plan" in state:
        print(f"   ✓ Plan created with {len(state['plan'])} steps")
    else:
        print("   ✗ Planning node failed")
        return

    # Step 3: Data loading node
    print("\n3. Executing data loading node...")
    data_result = data_loading_node(state)
    state = {**state, **data_result}

    if "mission" in state:
        print("   ✓ Data loaded successfully")
        print(f"   - Mission: {state['mission']['mission_name']}")
        print(f"   - Tasks: {len(state['mission_tasks'])}")
        print(f"   - Agents: {len(state['specialized_agents'])}")
        print(f"   - KPIs: {len(state['mission_kpis'])} keys")

        # Show task details
        print("\n   Task Details:")
        for task in state['mission_tasks']:
            agents = get_capable_agents_for_task(task['task_id'])
            agent_names = [get_agent_by_id(aid)['name'] for aid in agents if get_agent_by_id(aid)]
            print(f"   - {task['task_id']}: {task['task']}")
            print(f"     Order: {task['order']}, Depends on: {task['depends_on']}")
            print(f"     Agents: {', '.join(agent_names)}")
            print(f"     Duration: {task['estimated_duration_minutes']} min, Approval: {task['requires_human_approval']}")
    else:
        print("   ✗ Data loading node failed")
        if state.get("errors"):
            for error in state["errors"]:
                print(f"     - {error}")

    print("\n" + "=" * 60)
    print("Full Flow Test Complete!")
    print("=" * 60)


if __name__ == "__main__":
    try:
        test_utilities()
        test_data_loading_node()
        test_full_flow()
        print("\n✅ All tests completed successfully!")
    except Exception as e:
        print(f"\n❌ Error during testing: {e}")
        import traceback
        traceback.print_exc()



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python test_data_loading_standalone.py
============================================================
Testing Data Loading Utilities
============================================================

1. Loading business missions...
   ✓ Loaded 3 missions
   - M001: Reduce Customer Onboarding Time
   - M002: Accelerate Sales Pipeline Progression
   - M003: Improve Support Case Resolution

2. Getting mission M001...
   ✓ Found: Reduce Customer Onboarding Time
   Description: Optimize steps required to onboard new customers to shorten time-to-value.

3. Loading tasks for mission M001...
   ✓ Loaded 3 tasks
   - T1: Collect customer information (order: 1, depends_on: [])
   - T2: Verify documents (order: 2, depends_on: ['T1'])
   - T3: Schedule onboarding call (order: 3, depends_on: ['T2'])

4. Getting task T1...
   ✓ Found: Collect customer information
   Duration: 5 minutes
   Requires approval: False

5. Loading specialized agents...
   ✓ Loaded 5 agents
   - A1: Data Collection Agent
   - A2: Document Verification Agent
   - A3: Scheduling Agent
   - A4: Sales Analysis Agent
   - A5: Support Resolution Agent

6. Getting agent A1...
   ✓ Found: Data Collection Agent
   Description: Gathers structured customer information and validates completeness.

7. Loading capabilities matrix...
   ✓ Loaded 8 task-to-agent mappings
   - Task T1 → Agents: ['A1']
   - Task T2 → Agents: ['A2']
   - Task T3 → Agents: ['A3']

8. Getting capable agents for task T1...
   ✓ Task T1 can be performed by: ['A1']

9. Loading KPIs for mission M001...
   ✓ Loaded KPIs:
   - target_onboarding_time_days: 2
   - baseline_onboarding_time_days: 5
   - max_steps: 5

10. Validating data consistency...
   ✓ All data is consistent!

============================================================
Utility Tests Complete!
============================================================

============================================================
Testing Data Loading Node
============================================================

1. Testing with mission_id M001...
   ✓ Node executed successfully
   - Mission: Reduce Customer Onboarding Time
   - Tasks loaded: 3
   - Agents loaded: 5
   - Capabilities mappings: 8
   - KPIs loaded: 3 keys

2. Testing with invalid mission_id M999...
   ✓ Correctly handled invalid mission_id
   Error: data_loading_node: Mission M999 not found

3. Testing with missing mission_id...
   ✓ Correctly handled missing mission_id
   Error: data_loading_node: mission_id is required

============================================================
Node Tests Complete!
============================================================

============================================================
Testing Full Flow: Goal → Planning → Data Loading
============================================================

1. Executing goal node...
   ✓ Goal defined: Execute mission M001

2. Executing planning node...
   ✓ Plan created with 7 steps

3. Executing data loading node...
   ✓ Data loaded successfully
   - Mission: Reduce Customer Onboarding Time
   - Tasks: 3
   - Agents: 5
   - KPIs: 3 keys

   Task Details:
   - T1: Collect customer information
     Order: 1, Depends on: []
     Agents: Data Collection Agent
     Duration: 5 min, Approval: False
   - T2: Verify documents
     Order: 2, Depends on: ['T1']
     Agents: Document Verification Agent
     Duration: 10 min, Approval: True
   - T3: Schedule onboarding call
     Order: 3, Depends on: ['T2']
     Agents: Scheduling Agent
     Duration: 3 min, Approval: False

============================================================
Full Flow Test Complete!
============================================================

✅ All tests completed successfully!
